<a href="https://colab.research.google.com/github/Angeltrek/TC3006C-Project/blob/main/RETO_Labubus_Matcha_Late.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# IMPORTANT: SOME KAGGLE DATA SOURCES ARE PRIVATE
# RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES.
import kagglehub
kagglehub.login()


In [ ]:
# IMPORTANT: RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES,
# THEN FEEL FREE TO DELETE THIS CELL.
# NOTE: THIS NOTEBOOK ENVIRONMENT DIFFERS FROM KAGGLE'S PYTHON
# ENVIRONMENT SO THERE MAY BE MISSING LIBRARIES USED BY YOUR
# NOTEBOOK.

covid_segmentation_path = kagglehub.competition_download('covid-segmentation')

print('Data source import complete.')


In [ ]:

# IMPORTANT: RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES
# TO THE CORRECT LOCATION (/kaggle/input) IN YOUR NOTEBOOK,
# THEN FEEL FREE TO DELETE THIS CELL.
# NOTE: THIS NOTEBOOK ENVIRONMENT DIFFERS FROM KAGGLE'S PYTHON
# ENVIRONMENT SO THERE MAY BE MISSING LIBRARIES USED BY YOUR
# NOTEBOOK.

import os
import sys
from tempfile import NamedTemporaryFile
from urllib.request import urlopen
from urllib.parse import unquote, urlparse
from urllib.error import HTTPError
from zipfile import ZipFile
import tarfile
import shutil

CHUNK_SIZE = 40960
DATA_SOURCE_MAPPING = 'covid-segmentation:https%3A%2F%2Fstorage.googleapis.com%2Fkaggle-competitions-data%2Fkaggle-v2%2F20561%2F1187889%2Fbundle%2Farchive.zip%3FX-Goog-Algorithm%3DGOOG4-RSA-SHA256%26X-Goog-Credential%3Dgcp-kaggle-com%2540kaggle-161607.iam.gserviceaccount.com%252F20240906%252Fauto%252Fstorage%252Fgoog4_request%26X-Goog-Date%3D20240906T172045Z%26X-Goog-Expires%3D259200%26X-Goog-SignedHeaders%3Dhost%26X-Goog-Signature%3D43742a5ce41d83ccfc3c8743c5c8079abe9d6551dac4966a23f0bc0a359986061563265bf12ebfe02c15b74ea516fd390ae87498e2b0eeee4d0c4130ff222cb97c8180aec7776cdaa156a56afbb9f3bf08925e57e8ef81a3037b39d68d6aaff017bb0bbe4ccf290b729140201790b7e8858beec45fb63e06ff0e01a0104b2d197c79b64fbc94fc2b8466da932dfe6f4d07651cb02bf789876345aa01ebdc11e32c56c168e4cbc2cd8fd289562cc0e8657c10e99c48d53bf333bb326e02f09e13f8e67438790fc2787521404e3b6ce197637c4398bd5dd04f4f099e48afc7a8b2e837ea63c9aa7ff785a50612b2fadb12bdaca4ed058122abc50b1e866742e2bf'

KAGGLE_INPUT_PATH='/kaggle/input'
KAGGLE_WORKING_PATH='/kaggle/working'
KAGGLE_SYMLINK='kaggle'

!umount /kaggle/input/ 2> /dev/null
shutil.rmtree('/kaggle/input', ignore_errors=True)
os.makedirs(KAGGLE_INPUT_PATH, 0o777, exist_ok=True)
os.makedirs(KAGGLE_WORKING_PATH, 0o777, exist_ok=True)

try:
  os.symlink(KAGGLE_INPUT_PATH, os.path.join("..", 'input'), target_is_directory=True)
except FileExistsError:
  pass
try:
  os.symlink(KAGGLE_WORKING_PATH, os.path.join("..", 'working'), target_is_directory=True)
except FileExistsError:
  pass

for data_source_mapping in DATA_SOURCE_MAPPING.split(','):
    directory, download_url_encoded = data_source_mapping.split(':')
    download_url = unquote(download_url_encoded)
    filename = urlparse(download_url).path
    destination_path = os.path.join(KAGGLE_INPUT_PATH, directory)
    try:
        with urlopen(download_url) as fileres, NamedTemporaryFile() as tfile:
            total_length = fileres.headers['content-length']
            print(f'Downloading {directory}, {total_length} bytes compressed')
            dl = 0
            data = fileres.read(CHUNK_SIZE)
            while len(data) > 0:
                dl += len(data)
                tfile.write(data)
                done = int(50 * dl / int(total_length))
                sys.stdout.write(f"\r[{'=' * done}{' ' * (50-done)}] {dl} bytes downloaded")
                sys.stdout.flush()
                data = fileres.read(CHUNK_SIZE)
            if filename.endswith('.zip'):
              with ZipFile(tfile) as zfile:
                zfile.extractall(destination_path)
            else:
              with tarfile.open(tfile.name) as tarfile:
                tarfile.extractall(destination_path)
            print(f'\nDownloaded and uncompressed: {directory}')
    except HTTPError as e:
        print(f'Failed to load (likely expired) {download_url} to path {destination_path}')
        continue
    except OSError as e:
        print(f'Failed to load {download_url} to path {destination_path}')
        continue

print('Data source import complete.')


# Segmentación de imagenes para la detección del COVID-19

Esta solución esta basada los notebooks siguientes:
- [PyTorch Baseline for Semantic Segmentation](https://www.kaggle.com/code/maedemaftouni/pytorch-baseline-for-semantic-segmentation) hecho por Maede Maftouni.
- [covid-segmentation](https://www.kaggle.com/code/oskaradolfovillalpez/covid-segmentation) hecho por Oskar Adolfo Villa López.

Team 1:
* Angel Mauricio Ramirez Herrera - A01710158
* Diego Antonio García Padilla - A01710777
* José Eduardo Viveros Escamilla - A01710605
* Kevin Alejandro Ramírez Luna - A01711063
* Guadalupe Paulina López Cuevas - A01701095
* Cristian Chávez Guía - A01710680




# **INTRODUCCIÓN**

El propósito de este notebook es abordar una problematica crítica que es la segmentación de imágenes CT (tomografías computarizadas)  médicas relacionadas con el COVID-19. El objetivo es identificar y delimitar áreas especificas de los pulmones. Uno de los problemas más comunes en la segmentación es detectar caraterísticas clave como las opacidades en vidrio esmerilado y las consolidaciones, siendo estos indicadores importantes en las infecciones por COVID-19. Generar este tipo de segmentaciones en áreas precisas es fundamental para los diagnósticos médicos ya que esto permite evaluar tanto la gravedad de la infección como información valiosa de en que parte esta la enfermedad.

**COVID-19**

El COVID-19 ingresa al cuerpo principalmente por vías respiratorias dañando a las células alveolares, reduciendo la capacidad de oxigenación en la sangre.
Las imágenes CT (tomografías computarizadas) muestran patrones característicos:

- Opacidades en vidrio esmerilado (Ground-glass)
    - Hallazgo temprano: Áreas grises y difusas, representan inflamación leve o acumulación de líquido en alvéolos.
- Consolidaciones
    - Enfermedad más avanzada o grave: Áreas blancas y densas, representan acumulación de líquido y células inflamatorias.

Importancia de la segmentación en CT:

-	Cuantificar la extensión del daño pulmonar.
-	Monitorear la progresión de la enfermedad.
-	Evaluar la respuesta al tratamiento.
-	Asistir en el diagnóstico rápido.



**COMPETENCIA KAGGLE**

Objetivo

-	Desarrollar modelos de segmentación médica para detectar zonas infectadas en pulmones a partir de imágenes CT (tomografías computarizadas).

Datos disponibles

-	Imágenes CT
-	Máscaras anotadas por especialistas que delimitan áreas infectadas.

Propósito

-	Mejorar la precisión y velocidad del diagnóstico, apoyando  a radiólogos en la lucha contra la pandemia.



**IMPLICACIONES ÉTICAS**

Las implicaciones éticas en la segmentación de imágenes CT médicas

-	Privacidad y Confidencialidad:
    - Las imágenes contienen información sensible del paciente por lo que es crucial el anonimato de los datos, eliminando metadatos identificables, además de obtener el consentimiento informado para su uso en investigación cumpliendo regularizaciones:
        - GDPR (Europa):
        - HIPPA (EE.UU):
-	Sesgo en datos:
    - Es esencial usar datasets diversos y representativos.
-	Uso clínico vs experimental
    - Los modelos en fase de investigación no deben usarse clínicamente sin validación rigurosa. Aprobación ética y ensayos clínicos para garantizar seguridad y eficacia.

In [ ]:
!pip install segmentation_models_pytorch

In [ ]:

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All"
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# **ANÁLISIS DE DATOS**

**ETL/EDA**

El dataset proporcionado por la competencia incluye imágenes CT de pulmones con su respectiva máscara de segmentación. Las imágenes provienen de 2 datasets distintos:

**Medseg Dataset:**
  
- 100 imágenes CT de 40 pacientes
- Las respectivas máscaras de segmentación que contienen 4 canales:
    - Opacidades de vidrio esmerilado (ground glass)
    - Consolidaciones
    - Tejidos pulmonares no afectados (Lungs Other)
    - Fondo

**Radiopaedia Dataset:**

- 829 imágenes CT
- Misma estructura y mascaras que el dataset de Medseg

# **Extracción de datos**

Los datos y las máscaras (etiquetas con clase de cada imagen) son extraidos de los repositorios dados, donde las imagener se almacenan en un Numpy Array

In [ ]:
prefix = '/kaggle/input/covid-segmentation/'

images_radiopedia = np.load(os.path.join(prefix, 'images_radiopedia.npy')).astype(np.float32)
masks_radiopedia = np.load(os.path.join(prefix, 'masks_radiopedia.npy')).astype(np.int8)
images_medseg = np.load(os.path.join(prefix, 'images_medseg.npy')).astype(np.float32)
masks_medseg = np.load(os.path.join(prefix, 'masks_medseg.npy')).astype(np.int8)

test_images_medseg = np.load(os.path.join(prefix, 'test_images_medseg.npy')).astype(np.float32)

La función **visualize** nos permite ver una matriz donde se despliegan las imagenes originales (en escala de grises) en la fila superior. Cada una de las máscaras se depliega por cada columna correspondiente. En este caso las máscaras que queremos que el modelo prediga son las de ground glass y consolidation que se encuentran en la segunda y tercera fila respectivamente.

Cada imagen tiene dimensiones de 520x520 pixeles.
Las clases de las máscaras de segmentación son las siguientes:

-	Class 0 - "ground glass" - objetivo
-	Class 1 - "consolidations" - objetivo
-	Class 2 - "lungs other" - no importante
-	Class 3 - "background" - no importante

In [ ]:
def visualize(image_batch, mask_batch=None, pred_batch=None, num_samples=8, hot_encode=True):
    num_classes = mask_batch.shape[-1] if mask_batch is not None else 0
    fix, ax = plt.subplots(num_classes + 1, num_samples, figsize=(num_samples * 2, (num_classes + 1) * 2))

    for i in range(num_samples):
        ax_image = ax[0, i] if num_classes > 0 else ax[i]
        if hot_encode:
            ax_image.imshow(image_batch[i,:,:,0], cmap='Greys')
        else:
            ax_image.imshow(image_batch[i,:,:])
        ax_image.set_xticks([])
        ax_image.set_yticks([])

        if mask_batch is not None:
            for j in range(num_classes):
                if pred_batch is None:
                    mask_to_show = mask_batch[i,:,:,j]
                else:
                    mask_to_show = np.zeros(shape=(*mask_batch.shape[1:-1], 3))
                    mask_to_show[..., 0] = pred_batch[i,:,:,j].cpu().numpy() > 0.5
                    mask_to_show[..., 1] = mask_batch[i,:,:,j]
                ax[j + 1, i].imshow(mask_to_show, vmin=0, vmax=1)
                ax[j + 1, i].set_xticks([])
                ax[j + 1, i].set_yticks([])

    plt.tight_layout()
    plt.show()

In [ ]:
visualize(images_radiopedia[30:], masks_radiopedia[30:])

Como podemos observar en la segunda y tercera fila, las dos clases que deseamos segmentar son áreas muy pequeñas comparadas con la imagen completa, lo que significa que las predicciones seran más complejas de hacer.

# **Preprocesamiento**

En la función **onehot_to_mask** lo que hacemos es que debemos convertir las máscaras de segmentación de “one-hot” a formato “índice de clase”, es decir, cambiamos la máscara de 4 canales a un NumPy array, para mapear con colores cada clase.

In [ ]:
def onehot_to_mask(mask, palette):
    """
    Converts a mask (H, W, K) to (H, W, C)
    """
    x = np.argmax(mask, axis=-1)
    colour_codes = np.array(palette)
    x = np.uint8(colour_codes[x.astype(np.uint8)])
    return x

palette = [[0], [1], [2],[3]]
masks_radiopedia_recover = onehot_to_mask(masks_radiopedia, palette).squeeze()  # shape = (H, W)

masks_medseg_recover = onehot_to_mask(masks_medseg, palette).squeeze()  # shape = (H, W)

print('Hot encoded mask size: ',masks_radiopedia.shape)
print('Paletted mask size:',masks_medseg_recover.shape)

visualize(masks_medseg_recover[30:],hot_encode=False)

# **Dataset Train, Validation y Test**

Ahora debemos separar los datasets en train y validation, donde para validation tomaremos 24 imagenes del dataset de Medseg, todas las demas imágenes restantes de Medseg y todas las de radiopedia seran para entrenamiento siendo un aprox. de ~800.

Para el detaset de test, estas son proporcionadas por la competencia de kaggle.

In [ ]:
masks_radiopedia_recover = onehot_to_mask(masks_radiopedia, palette).squeeze()  # shape = (H, W)
masks_medseg_recover = onehot_to_mask(masks_medseg, palette).squeeze()  # shape = (H, W)


val_indexes, train_indexes = list(range(24)), list(range(24, 100))

train_images = np.concatenate((images_medseg[train_indexes], images_radiopedia))
train_masks = np.concatenate((masks_medseg_recover[train_indexes], masks_radiopedia_recover))
val_images = images_medseg[val_indexes]
val_masks = masks_medseg_recover[val_indexes]

batch_size = len(val_masks)

del masks_medseg_recover
del masks_radiopedia_recover
del images_radiopedia
del masks_radiopedia
del images_medseg
del masks_medseg

Previamente al entrenamiento del modelo, debido a que el dataset de entrenamiento es menor a ~1000 imágenes, para poder mejorar la variabilidad de imagenes se realizaron las siguientes transformaciones para el dataset de train:

-	**Redimensionamiento**: Cambiar las dimensiones de las imágenes a 256x256 pixeles para tener consistencia en las dimensiones de entrada.
-   **Rotaciones**: Con un limite de 360° y con una probabilidad del 90%, para que las imagenes pueda llegar a estar totalmente rotadas.
-   **Flip horizontal**: Con una probabilidad el 50%, para voltear de manera horizontal las imágenes.

Con estas transformaciones podemos asegurar que el modelo pueda predecir diferentes variaciones, haciéndolo más robusto.

In [ ]:
import albumentations

import cv2

SOURCE_SIZE = 512
TARGET_SIZE = 256


train_augs = albumentations.Compose([
    albumentations.Rotate(limit=360, p=0.9, border_mode=cv2.BORDER_REPLICATE),
    albumentations.RandomSizedCrop((int(SOURCE_SIZE * 0.75), SOURCE_SIZE),
                                   (TARGET_SIZE, TARGET_SIZE),
                                   interpolation=cv2.INTER_NEAREST),
    albumentations.HorizontalFlip(p=0.5),

])

val_augs = albumentations.Compose([
    albumentations.Resize(TARGET_SIZE, TARGET_SIZE, interpolation=cv2.INTER_NEAREST)
])

# **Load Dataset**

Para la creación del objeto **Dataset**, creamos tres métodos además del constructor:

- **__getitem__**: Nos devuelve un elemento del dataset despues de transformar la imagen a un tensor y también aplica la tranformación y normalización de las imagenes.
- **__len__**: Devuelve la longitud del dataset
- **tiles**: Devuelve dos tensors que contienen una imagen y su máscara, donde cada elemento corresponde a un parche de la imagen orginal.

In [ ]:
from PIL import Image
import segmentation_models_pytorch as smp
import torch.nn.functional as F
import torch.nn as nn
from tqdm.notebook import tqdm
import time

from torchvision import transforms as T
from torch.utils.data import Dataset, DataLoader
import torch

class Dataset:
    def __init__(
            self,
            images,
            masks,
            augmentations=None
    ):
        self.images = images
        self.masks = masks
        self.augmentations = augmentations
        self.mean = [0.485]
        self.std = [0.229]

    def __getitem__(self, i):
        image = self.images[i]
        mask = self.masks[i]


        if self.augmentations is not None:
            sample = self.augmentations(image=image, mask=mask)

            image, mask = Image.fromarray(np.squeeze(sample['image'], axis=2)), sample['mask']

        if self.augmentations is None:
            image = Image.fromarray(image)

        t = T.Compose([T.ToTensor(), T.Normalize(self.mean, self.std)])
        image = t(image)
        mask = torch.from_numpy(mask).long()

        return image, mask

    def __len__(self):
        return len(self.images)

    def tiles(self, image, mask):

        img_patches = image.unfold(1, 512, 512).unfold(2, 768, 768)
        img_patches  = img_patches.contiguous().view(3,-1, 512, 768)
        img_patches = img_patches.permute(1,0,2,3)

        mask_patches = mask.unfold(0, 512, 512).unfold(1, 768, 768)
        mask_patches = mask_patches.contiguous().view(-1, 512, 768)

        return img_patches, mask_patches


train_dataset = Dataset(train_images, train_masks, train_augs)
val_dataset = Dataset(val_images, val_masks, val_augs)

train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_dataloader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)

# **Funciones para visualización y análisis**

Antes de definir el modelo, necesitamos funciones que ayuden a visualizar y analizar el rendimiento del modelo.

En la función **mask_to_onehot** convertimos la máscara de segmentación a One_hot encoded. Esto es una representación binaria donde cada clase se representa con un vector que tiene valor de 1 en la posición correspondiente a la clase y 0 en todas las demás.

- Crea una lista vacia **sematic_map** para almacenar los mapas de clases de cada color.
- Para cada uno, la máscara se compara con dicho color, generando una matriz booleana que indica que pixel coincide con ese color. Se verifica así todos los canales de color coinciden, generando un mapa binario, True el pixel pertenece a la clase que corresponde al color.
- Los mapas de clase para cada color se apilan en una nueva dimesión, donde **semantic_map** se convierte en una matriz de dimensiones (H, W, K), K es el num de clases.
- La función regresa un tensor donde cada pixel esta representado por un vector one-hot que indica su clase.




In [ ]:
def mask_to_onehot(mask, palette):
    """
    Converts a segmentation mask (H, W, C) to (H, W, K) where the last dim is a one
    hot encoding vector, C is usually 1 or 3, and K is the number of class.
    """
    semantic_map = []
    for colour in palette:
        #print('colour',colour)
        equality = np.equal(mask, colour)
        #print('equality',equality)
        class_map = np.all(equality, axis=-1)
        semantic_map.append(class_map)
    semantic_map = np.stack(semantic_map, axis=-1).astype(np.float32)
    return torch.from_numpy(semantic_map)

i,train_data = next(enumerate(train_dataloader))

mask_hot_encoded = mask_to_onehot(torch.unsqueeze(train_data[1],-1).numpy(),palette)
#visualize(torch.unsqueeze(torch.squeeze(train_data[0],1),-1),mask_hot_encoded)
visualize(train_data[0].permute(0, 2, 3,1),mask_hot_encoded)


**PIXEL ACCURACY**

En la función **pixel_accuracy** lo que calculamos es la exactitud por pixel para la máscara de segmentación predicha comparada con la máscara de referencia (ground truth). Esta métrica mide el porcentaje de pixeles en la imagen que coinciden adecuadamente con su clasificación, es decir, si la etiqueta predicha es igual a la etiqueta de referencia.

- La salida del modelo la pasamos por **softmax** para convertirla en probabilidades de clases en cad posición de pixel.
- Seleccionamos la clase con mayor probabilidad usando **torch.argmax*
- Se compara la máscara predicha con la máscara de referencia y se genera una matriz binaria:
    - 1 prediccion correcta
    - 0 prediccion incorrecta.
- Se suman todas las predicciones correctas y se dividen entre el núm total de pixeles en la máscara para calcular la exactitud por pixel.
- La función devuelve un valor float.



In [ ]:
def pixel_accuracy(output, mask):
    with torch.no_grad():
        output = torch.argmax(F.softmax(output, dim=1), dim=1)
        correct = torch.eq(output, mask).int()
        accuracy = float(correct.sum()) / float(correct.numel())
    return accuracy

**INTERSECCIÓN SOBRE LA UNIÓN (ioU)**

Esta función calcula la **Intersección sobre la Unión (ioU)** entre la mascara predicha y la mascara de referencia, con el fin de medir la superposición entre las etiquetas predichas y las etiquetas reales de cada clase.
- La máscara predicha y la máscara de referencia se aplanan, se mueven al CPU y se convierten en Numpy arrays.
- Convierte la máscara predicha en un formato de etiqueta usando **ToLabel()**.
- Se crea una matriz nueva (agg) sumando la máscara predicha y la de referencia. Cada pixel representa una combinación de la clase predicha y la real.
- Se calcula el núm de pixeles donde predicción y real son 1, representando la intersección (i).
- La unión (u) se calcula como el num de pixeles donde predicción y real son mayor que 0, representando donde la clase fue predicha o esta presente en la referencia.
    - Si u==0, la IoU se establece como 1 (coincidencia perfecta)
    - De lo contrario, IoU se calcula como i/u


In [ ]:
def iou(pred, gt):
    pred = pred.squeeze().cpu().data.numpy()
    pred = ToLabel(pred)
    gt = gt.squeeze().cpu().data.numpy()
    agg = pred + gt
    i = float(np.sum(agg == 2))
    u = float(np.sum(agg > 0))
    if u == 0:
        result = 1
    else:
        result = i/u
    return result

**MEDIA DE LA INTERSECCIÓN SOBRE LA UNIÓN (mIoU)**

El **Mean Intersection over Union (mIoU)** es una métrica que calcula el promedio de la **Intersección sobre Union (ioU)** entre etiquetas predichas y etiquetas reales.

- La salida del modelo la pasamos por softmax para convertirla en probabilidades de clases en cad posición de pixel.
- Seleccionamos la clase con mayor probabilidad usando torch.argmax para crear la máscara predicha.
- La máscara predicha y la de referencia se transforman el vetores unidimensionales.
- Se inicializa una lista para almacenar el ioU de cada clase.
- La función identifica los pixeles de la máscara predicha que pertenecen a la clase actual y los pixeles en la máscara de referencia.
    - Si no hay pixeles de la clase actual en la máscara de referencia, se agrega un np.nan a la lista de ioU para esa clase.
    - Caso contrario, se calcula la intersección y la unión.
- La ioU para esa clase se saca dividiendo la intersección entre la unión.
- La función devuelve el promedio de ioU par todas las clases, sin contar los nan.


In [ ]:
def mIoU(pred_mask, mask, smooth=1e-10, n_classes=4):
    with torch.no_grad():
        pred_mask = F.softmax(pred_mask, dim=1)
        pred_mask = torch.argmax(pred_mask, dim=1)
        pred_mask = pred_mask.contiguous().view(-1)
        mask = mask.contiguous().view(-1)

        iou_per_class = []
        for clas in range(0, n_classes): #loop per pixel class
            true_class = pred_mask == clas
            true_label = mask == clas

            if true_label.long().sum().item() == 0: #no exist label in this loop
                iou_per_class.append(np.nan)
            else:
                intersect = torch.logical_and(true_class, true_label).sum().float().item()
                union = torch.logical_or(true_class, true_label).sum().float().item()

                iou = (intersect + smooth) / (union +smooth)
                iou_per_class.append(iou)
        return np.nanmean(iou_per_class)

# **Arquitectura del Modelo**

En este caso se opto por usar variantes de la arquitectura U-Net:

Efficientnet-B2 esta diseñada par lograr un equilibrio óptimo entre eficiencia computacional y precisión, utiliza una técnica llamada “compound scaling” para ajustar la profundidad (núm de capas), el ancho y la resolución del modelo.

U-net++ es una evolución de u-Net que mejora la segmentación de estructuras médicas complejas. Debido a sus conexiones anidadas y densas esto permite una mejor integración de características de diferentes escalas. Es ideal para imágenes CT donde los detalles finos son críticos.

-	Encoder (Downsampling):
    - Extrae características mediante bloques convolucionales y reduce la resolución espacial.
    - 4 niveles de downsampling
-	Decoder (Upsampling):
    - Recupera resolución original combinando características den encoder y las conexiones densas.
    - 4 niveles de upsampling
-	Conexiones anidadas:
    - Conexiones densas entre todos los bloques del mismo nivel.

**Detalles de EfficientNet-B2 (ENCODER)**:

Funciona como “extractor de caracteristicas”, utiliza una técnica llamada “escalado compuesto” que ajusta simultaneamente:
- Profundidad (núm de capas)
- Ancho (núm de filtros)
- Resolución (tamaño de entrada)

Mira detalles finos (bordes, texturas pequeñas) y a medida que baja el nivel detecta formas grandes (órganos completos). En vez de tijeras grandes (Max pooling), convoluciones inteligentes para reducir el tamaño.

-	Características:
    - Número de capas: 16 capas convolucionales
    - Kernel size: 3x3
    - Stride: capas iniciales 1 (reducción rápida de dimensionalidad), capas posteriores 2 (mantener resolución durante extracción profunda)
    - Padding: usa “same” en la mayoría de las capas, tamaño de salida permanece igual que el de la entrada.
    - Max Pooling: No usa max pooling, se basa en stride para conservar resolución, mejora eficiencia computacional.
    - Activación: ReLU

**Detalles de U-Net++ (DECODER)**:

-	Características:
    - Número de capas: 5 por rama de decoder
    - Kernel size: 3x3 en todas las capas
    - Stride: 1
    - Padding: Usa padding “same”, el tamaño de salida permanece igual al de la entrada, conserva resolución.
    - Upsampling: Se usa transposed convolution, kernel 2x2 y stride de 2
    - Skip Connections: Pasan por bloques convolucionales adicionales para alienar características antes de fusionarlas.

**Capas finales:**

-	Activación:
    - Softmax, multiclase exclusivo

**Función de perdida**

-	Cross-Entropy Loss:
    - Penaliza predicciones incorrectas por pixel


Los puentes tienen capas intermedias de convolucion adicionales: limpia y refina la informacion antes de fusionarla.

In [ ]:
model = smp.UnetPlusPlus('efficientnet-b2',in_channels=1, encoder_weights='imagenet',classes=4, activation=None, encoder_depth=5, decoder_channels=[256, 128, 64, 32, 16], decoder_use_batchnorm=True)
#decoder_attention_type ='scse'

# **FUNCIONES DE ENTRENAMIENTO**

**Función de entrenamiento**: Definimos la función de entrenamiento, donde además de entrenar también evalúa el modelo con un conjunto de datos (train) durante un número específico de epochs.

**Función de pérdida**: Criterion representa la función de perdida que se utilizara para entrenar y ajustar los pesos del modelo con Cross Entropy.

**Optimizador, Learning rate y Scheduler:**

-	Optimizer, recibe optimizer como parámetro, puede ser Adam o SGD, ajusta los pesos del modelo basándose en los gradientes, en este caso usaremos Adam.
-	Learning rate y Scheduler, se controla por el optimizador y se va a ajustar dinámicamente con un scheduler. La función get_lr() obtiene el learning rate actual y lo almacena en una lista.

**Numero de epochs y tamaño de minibatch:**

-	Numero de epochs define cuantas veces el modelo entrenara sobre un conjunto de datos completos.
-	Minibatch, El tamaño de minibatch se determinaran usando train_loader y  val_loader, se define el tamaño al crear dataLoader.

**Técnica de regularización:**

-	Early stopping: Si la pérdida de val no mejora después de 7 epochs seguidas, se detiene el entrenamiento, evitando overfitting (sobreajuste)
-	Guardado del mejor modelo: El modelo se guarda cada que la perdidad de val disminuya, con esto garantizamos guardar el mejor modelo.


# **PROCESO DE ENTRENAMIENTO**

**Entrenamiento:**

-	El modelo se pone en entrenamiento con model.train().
-	Cada minibatch de imágenes y máscaras pasan por el modelo
-	Se calcula la perdida y se hace back propagation con los.backward()
-	El optimizador actualiza los pesos con optimizar.step()
-	El Scheduler ajusta el learning rate con scheduler.step()

**Validación:**

-	Después de cada epoch, el modelo se evalua en el dataset de validation con model.eval() sin actualizar pesos.
-	Se calculan las méttricas mIoU y pixel accuracy para ambos datasets.


In [ ]:
def get_lr(optimizer):
    for param_group in optimizer.param_groups:
        return param_group['lr']

def fit(epochs, model, train_loader, val_loader, criterion, optimizer, scheduler, patch=False):
    #torch.cuda.empty_cache()
    train_losses = []
    test_losses = []
    val_iou = []; val_acc = []
    train_iou = []; train_acc = []
    lrs = []
    min_loss = np.inf
    decrease = 1 ; not_improve=0

    model.to(device)
    fit_time = time.time()
    for e in range(epochs):
        since = time.time()
        running_loss = 0
        iou_score = 0
        accuracy = 0
        #training loop
        model.train()
        for i, data in enumerate(tqdm(train_loader)):
            #training phase
            image_tiles, mask_tiles = data

            image = image_tiles.to(device); mask =mask_tiles.to(device);
            #forward
            output = model(image)

            loss = criterion(output, mask)
            #evaluation metrics
            iou_score += mIoU(output, mask)
            accuracy += pixel_accuracy(output, mask)
            #backward
            loss.backward()
            optimizer.step() #update weight
            optimizer.zero_grad() #reset gradient

            #step the learning rate
            lrs.append(get_lr(optimizer))
            scheduler.step()

            running_loss += loss.item()


        else:
            model.eval()
            test_loss = 0
            test_accuracy = 0
            val_iou_score = 0
            #validation loop
            with torch.no_grad():
                for i, data in enumerate(tqdm(val_loader)):
                    image_tiles, mask_tiles = data

                    image = image_tiles.to(device); mask =mask_tiles.to(device);
                    output = model(image)
                    #evaluation metrics
                    val_iou_score +=  mIoU(output, mask)
                    test_accuracy += pixel_accuracy(output, mask)
                    #loss
                    loss = criterion(output, mask)
                    test_loss += loss.item()

            #calculatio mean for each batch
            train_losses.append(running_loss/len(train_loader))
            test_losses.append(test_loss/len(val_loader))


            if min_loss > (test_loss/len(val_loader)):
                print('Loss Decreasing.. {:.3f} >> {:.3f} '.format(min_loss, (test_loss/len(val_loader))))
                min_loss = (test_loss/len(val_loader))
                decrease += 1
                if decrease % 5 == 0:
                    print('saving model...')
                    torch.save(model, 'Unet_efficientnet_b2_mIoU-{:.3f}.pt'.format(val_iou_score/len(val_loader)))


            if (test_loss/len(val_loader)) > min_loss:
                not_improve += 1
                min_loss = (test_loss/len(val_loader))
                print(f'Loss Not Decrease for {not_improve} time')
                if not_improve == 7:
                    print('Loss not decrease for 7 times, Stop Training')
                    break

            #iou
            val_iou.append(val_iou_score/len(val_loader))
            train_iou.append(iou_score/len(train_loader))
            train_acc.append(accuracy/len(train_loader))
            val_acc.append(test_accuracy/ len(val_loader))
            print("Epoch:{}/{}..".format(e+1, epochs),
                  "Train Loss: {:.3f}..".format(running_loss/len(train_loader)),
                  "Val Loss: {:.3f}..".format(test_loss/len(val_loader)),
                  "Train mIoU:{:.3f}..".format(iou_score/len(train_loader)),
                  "Val mIoU: {:.3f}..".format(val_iou_score/len(val_loader)),
                  "Train Acc:{:.3f}..".format(accuracy/len(train_loader)),
                  "Val Acc:{:.3f}..".format(test_accuracy/len(val_loader)),
                  "Time: {:.2f}m".format((time.time()-since)/60))

    history = {'train_loss' : train_losses, 'val_loss': test_losses,
               'train_miou' :train_iou, 'val_miou':val_iou,
               'train_acc' :train_acc, 'val_acc':val_acc,
               'lrs': lrs}
    print('Total time: {:.2f} m' .format((time.time()- fit_time)/60))
    return history

**CONSIDERACIONES DE HARDWARE**

Para el entrenamiento se uso la GPU para hacer el proceso mucho mas rápido, paralelizando los procesos computacionales. En este caso para los entrenamientos hechos en Kaggle se uso el acelerador de GPU P100.

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print('Using device:', device)
print()

# **Métricas de Evaluación**

**ENCONTRAR LEARNING RATE**

Para encontrar el mejor learning rate el modelo usa diferentes valores desde $ 1 \times 10^{-8} $ to 10. Esta técnica es inspirada en un artículo publicado por Leslie N. Smith, titulado *Cyclical Learning Rates for Training Neural Networks* (Tasas de Aprendizaje Cíclicas para Entrenar Redes Neuronales). Los valores de la perdida se almacenan en cada iteración usando una función para reducir el ruido.

In [ ]:
def find_lr(model, train_loader, criterion, optimizer, init_value=1e-8, final_value=10.0, beta=0.98):
    num_iter = len(train_loader)
    lr_values = []
    losses = []
    avg_loss = 0
    best_loss = float('inf')

    model.to(device)

    for i, (images, masks) in enumerate(tqdm(train_loader)):
        images = images.to(device)
        masks = masks.to(device)

        lr = init_value * (final_value / init_value) ** (i / num_iter)
        optimizer.param_groups[0]['lr'] = lr
        lr_values.append(lr)

        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, masks)
        avg_loss = beta * avg_loss + (1 - beta) * loss.item()
        smoothed_loss = avg_loss / (1 - beta ** (i + 1))
        losses.append(smoothed_loss)

        if smoothed_loss > 4 * best_loss or torch.isnan(loss):
            return lr_values, losses

        if smoothed_loss < best_loss:
            best_loss = smoothed_loss

        loss.backward()
        optimizer.step()

    return lr_values, losses

def plot_lr_finder(lr_values, losses):
    """
    Plots the learning rate finder results.

    Args:
        lr_values: A list of learning rates.
        losses: A list of losses.
    """
    plt.plot(lr_values, losses)
    plt.xscale('log')
    plt.xlabel('Learning Rate')
    plt.ylabel('Loss')
    plt.title('Learning Rate Finder')
    plt.show()

model_lr = smp.UnetPlusPlus('efficientnet-b2',in_channels=1, encoder_weights='imagenet',classes=4, activation=None, encoder_depth=5, decoder_channels=[256, 128, 64, 32, 16], decoder_use_batchnorm=True)
#decoder_attention_type ='scse'
weight_decay = 1e-4
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.AdamW(model_lr.parameters(), weight_decay=weight_decay)
# Find the optimal learning rate
lr_values, losses = find_lr(model_lr, train_dataloader, criterion, optimizer)

# Plot the results
plot_lr_finder(lr_values, losses)


**CAMBIO DE LA PERDIDA & LEARNING RATE**

Para visualizar de mejor manera los datos podemos usar la función de cambio de la perdida. El objetivo es encontrar el punto donde la perdidad decrece lo más posible, que es donde sacaremos el mayor learning rate.

In [ ]:
# Calculate the derivative of the loss
diff = np.diff(losses)

# Adjust lr_values to match the length of diff
adjusted_lr_values = lr_values[1:]

# Plot the derivative
plt.plot(adjusted_lr_values, diff)
plt.xscale('log')
plt.xlabel('Learning Rate')
plt.ylabel('Change of Loss')
plt.title('Rate of change of Loss')
plt.show()

# Find the index of the minimum derivative
min_diff_index = np.argmin(diff)

# Find the corresponding learning rate
min_diff_lr = adjusted_lr_values[min_diff_index]

# Print the learning rate corresponding to the lowest derivative of the loss
print("Learning Rate corresponding to the highest negative change of the loss:", min_diff_lr)


# **ENTRENAMIENTO**

El mejor learning rate ya fue encontrado con la funcion anterior. El número de epochs fue encontrado a travez de prueba y error, 15 fue el valor donde el overfitting (sobreajuste) fue el mínimo y las métricas las más altas. Los pesos no se modificaron con respecto al notebook inicial.

In [ ]:
max_lr = 3e-2
epoch = 15
weight_decay = 1e-4

criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.AdamW(model.parameters(), lr=max_lr, weight_decay=weight_decay)
sched = torch.optim.lr_scheduler.OneCycleLR(optimizer, max_lr, epochs=epoch,
                                            steps_per_epoch=len(train_dataloader))

history = fit(epoch, model, train_dataloader, val_dataloader, criterion, optimizer, sched)

In [ ]:
torch.save(model, 'Unet-efficientnet.pt')

# **Resultados del entrenamiento**

Después del entrenamiento del modelo, es importante ver las gráficas con respecto a la perdida, la media de ioU y la accuracy, que nos ayudan a saber como fue el comportamiento del entrenamiento y el ajuste que fue teniendo durante cada epoch.

In [ ]:
def plot_loss(history):
    plt.plot(history['val_loss'], label='val', marker='o')
    plt.plot( history['train_loss'], label='train', marker='o')
    plt.title('Loss per epoch'); plt.ylabel('loss');
    plt.xlabel('epoch')
    plt.legend(), plt.grid()
    plt.show()

def plot_score(history):
    plt.plot(history['train_miou'], label='train_mIoU', marker='*')
    plt.plot(history['val_miou'], label='val_mIoU',  marker='*')
    plt.title('Score per epoch'); plt.ylabel('mean IoU')
    plt.xlabel('epoch')
    plt.legend(), plt.grid()
    plt.show()

def plot_acc(history):
    plt.plot(history['train_acc'], label='train_accuracy', marker='*')
    plt.plot(history['val_acc'], label='val_accuracy',  marker='*')
    plt.title('Accuracy per epoch'); plt.ylabel('Accuracy')
    plt.xlabel('epoch')
    plt.legend(), plt.grid()
    plt.show()

In [ ]:
plot_loss(history)
plot_score(history)
plot_acc(history)

# **Evaluación de los resultados del entrenamiento**

**Loss:** The first graph shows the behavior of the loss over epochs for both training and validation. The loss seems extremely high in the first epoch of validation and then drops almost to zero. This indicates that the model quickly adjusts to the images.

**mIoU:**

**Accuracy:**

**mIoU (Mean Intersection over Union)**: The second graph shows the mIoU, which measures the overlap between the predicted and ground truth areas for segmentation. This metric is particularly important in image segmentation tasks because it is more robust for evaluating the overlap between the segmented and actual areas. Both training and validation sets have some variation before the 8th epoch, where both start to increase steadily.

**Accuracy**: The third graph displays the model’s accuracy, measuring the percentage of correctly segmented pixels. The results show a small gap between training accuracy. Although accuracy is useful, it is not ideal for segmentation tasks where the background class (non-lesion) might dominate the evaluation.

# **Resultados**

Creamos la funcion **predict_image_mask_miou** para cambiar el modelo a modo de evaluación y detener el entrenamiento, para probar las predicciones del modelo, de igual forma calcula los scores de mIoU y regresa la máscara de prediccion y el score final.

In [ ]:
def predict_image_mask_miou(model, image, mask, mean=[0.485], std=[0.229]):
    model.eval()
    model.to(device); image=image.to(device)
    mask = mask.to(device)


    with torch.no_grad():
        output = model(image)
        a,b,c,d = output.shape
        score = mIoU(output, mask)
        masked = torch.argmax(output, dim=1)
        masked = masked.cpu().squeeze(0)
    return masked, score, output.permute(0, 2, 3,1)




La funcion **predict_image_mask_pixel** sirve para algo similar a la anterior, pero en vez de calcular los scores de mIoU, revisa cuantos pixeles de manera individual fueron predichos de manera correcta usando **pixel_accuracy**

In [ ]:
def predict_image_mask_pixel(model, image, mask, mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]):
    model.eval()
    model.to(device); image=image.to(device)
    mask = mask.to(device)
    with torch.no_grad():

        output = model(image)
        acc = pixel_accuracy(output, mask)
        masked = torch.argmax(output, dim=1)
        masked = masked.cpu().squeeze(0)
    return masked, acc


La función **mask_to_onehot** convierte la máscara de colores en un one-hot encoding.

In [ ]:
def mask_to_onehot(mask, palette):
    """
    Converts a segmentation mask (H, W, C) to (H, W, K) where the last dim is a one
    hot encoding vector, C is usually 1 or 3, and K is the number of class.
    """
    semantic_map = []
    for colour in palette:
        equality = np.equal(mask, colour)
        class_map = np.all(equality, axis=-1)
        semantic_map.append(class_map)
    semantic_map = np.stack(semantic_map, axis=-1).astype(np.float32)
    return torch.from_numpy(semantic_map)

Usando las funciones anteriores, obtenemos un batch de imágenes con sus máscaras del dstaset de validación y pasamos las imagenes a travez de modelo para predecir su máscara. También sacamos su score mIoU para saber que tan bien las máscaras predichas coinciden con las máscaras reales y convertir la máscara de referencia a one-hot encoding.

In [ ]:
image, mask = next(iter(val_dataloader))

In [ ]:
pred_mask, score,output = predict_image_mask_miou(model, image, mask)
semantic_map = mask_to_onehot(torch.unsqueeze(mask,-1).cpu().numpy(),palette)

# **RESULTADOS VISUALES**

In [ ]:
visualize(image, semantic_map, pred_batch=output)

**INTERPRETACIÓN DE LOS GRÁFICOS**

-	Amarillo: Pixeles donde tanto la máscara de predicción como la máscara verdadera están activadas para una clase específica. El modelo predijo correctamente para esos pixeles.
-	Rojo: Pixeles donde el modelo predice una clase pero la máscara verdadera es negativa. El modelo predijo incorrectamente que estos pixeles pertenecen a una clase cuando en realidad no.
-	Verde: Pixeles donde la máscara de verdad si dice que pertenecen a una clase pero la predicción no identifico que esos pixeles correspondían a una clase.

Podemos observar que una gran parte de los pixeles esta en amarillo lo que significa que los valores que fueron correctamente predichos con respecto a la máscara de referencia, por otro lado tambien observamos que existen muchos pixeles en rojo lo que significa que hay muchos falsos positivos en las predicciones del modelo.

# **USO DEL MODELO EN DATASET DE PRUEBA**

Después de analizar el desempeño del modelo creamos la función **miou_score** para cad imagen del dataset de validation y test. Es un loop que pasa por el dataset y predice las máscaras de clases y calcula su score guardandolo en una lista que nos va a permitir ver el desempeño del modelo en el dataset del test.

In [ ]:
def miou_score(model, test_set):
    score_iou = []
    for i, data in enumerate(tqdm(test_set)):
        img, mask = data
        pred_mask, score,output = predict_image_mask_miou(model, img, mask)
        score_iou.append(score)
    return score_iou

mob_miou = miou_score(model, val_dataloader)
mob_miou



Limpiamos los datos de entrenamiento

In [ ]:
del train_images
del train_masks

**TEST PREDICT**

Creamos la funcion **test_predict** para normalizar y predecir la máscara de cada una de las imágenes del dataset de test. Usamos la función de **softmax** para normalizar el output de probabilidades y redimencionamos la máscara de predicción para que tenga las mismas dimensiones esperadas.

In [ ]:
def test_predict(model, image, mean=[0.485], std=[0.229]):
    t = T.Compose([T.ToTensor(), T.Normalize(mean, std)])
    image = t(image)
    model.eval()

    model.to(device); image=image.to(device)

    with torch.no_grad():
        output = model(torch.unsqueeze(image,1))
        output = nn.Softmax(dim=1)(output)
    return output.permute(0, 2, 3,1)

**VISUALIZACION DE RESULTADOS DE PRUEBA**

Procesamos un batch de imagenes del dataset de test y sus máscaras de predicción, las convertimos de tensors a arrays. Las máscaras predichas tienen un umbral de 0,5 y se muestran junto con las imágenes de entrada.

In [ ]:
image_batch = np.stack([val_augs(image=img)['image'] for img in test_images_medseg], axis=0)
print(torch.from_numpy(image_batch).shape)
print(image_batch[i].shape)
#output = test_predict(model, torch.from_numpy(image_batch).permute(0, 3, 1,2))
output = np.zeros((10,256,256,4))
for i in range(10):
    output[i] = test_predict(model, image_batch[i]).cpu().numpy()
print(output.shape)
test_masks_prediction = output > 0.5
visualize(image_batch, test_masks_prediction, num_samples=len(test_images_medseg))

Finalmente redimencionamos las máscaras predichas usando scipy, para agrandarlas para que coincidan con el tamaño o la resolución de la imagen original.

In [ ]:
import scipy
test_masks_prediction_original_size = scipy.ndimage.zoom(test_masks_prediction[..., :-2], (1, 2, 2, 1), order=0)
test_masks_prediction_original_size.shape

Aplanamos las máscaras predichas a una matriz unidimensional, transformamos los datos en un DataFrame y lo guardamos como un archivo CSV, el contenido del archivo tendrá el índice y los valores de máscara predichos para cada píxel.

In [ ]:
import pandas as pd

pd.DataFrame(
             data=np.stack((np.arange(len(test_masks_prediction_original_size.ravel())),
                            test_masks_prediction_original_size.ravel().astype(int)),
                            axis=-1),
             columns=['Id', 'Predicted'])\
.set_index('Id').to_csv('sub.csv')

# **Conclusión**

El modelo desarrollado demostró una capacidad notable para identificar regiones infectadas en imágenes CT de pulmones, segmentando áreas extensas y también pequeñas para un diagnóstico médico inicial. Los resultados visuales y cuantitativos confirman que la arquitectura U-Net++ con EfficientNet-B2 podria ser adecuada para esta tarea, capturando características clave como opacidades en vidrio esmerilado y consolidaciones.

Sin embargo, se observaron algunas limitaciones:

- **Falsos positivos (áreas rojas)**: El modelo en ocasiones marca como infectadas regiones sanas, posiblemente por un sobreajuste (overfitting) a patrones específicos del dataset de train o a similitudes visuales entre tejido sano e infectado.
- **Falsos negativos (áreas verdes)**: Aunque menos frecuentes, el modelo omite algunas áreas infectadas, especialmente cuando son pequeñas o presentan baja visibilidad.

Estos errores significan que es necesario mejorar la generalización del modelo y refinar su sensibilidad a características sutiles.

**Posibles mejoras:**

- Ajustar la loss function para enfocarse mejor en clases minoritarias (lesiones pequeñas).
- Incorporar técnicas de post-procesamiento para reducir falsos positivos.
- Buscar métodos de aumento de datos más avanzados para más diversidad y balance.

En general, el modelo tiene potencial para ser una buena herramienta para la segmentación de imágenes CT de COVID-19, pero requiere optimización para minimizar errores y garantizar su confiabilidad en entornos clínicos reales.

# **REFERENCIAS**

- Società Italiana di Radiologia Medica e Interventistica. (s.f.). Home. SIRM. Recuperado 14 de septiembre de 2024, de
https://sirm.org/en/

- Gaillard, F., & Weerakkody, Y. (s.f.). *COVID-19*. Radiopaedia.org. Recuperado 14 de septiembre de 2024, de
https://radiopaedia.org/articles/covid-19-4?lang=us

- Ronneberger, O., Fischer, P., & Brox, T. (2015). U-Net: Convolutional Networks for Biomedical Image Segmentation. arXiv.
https://doi.org/10.48550/arXiv.1506.01186

- GeekforGeeks. (s.f.). U-Net Architecture Explained. Recuperado 14 de septiembre de 2024, de
https://www.geeksforgeeks.org/machine-learning/unet-architecture-explained/